In [73]:
import pandas as pd

從原始檔可以看出第一個row是column的英文，為無效資料，故先行刪除，以利後續分析

In [74]:
path = 'C:\\Users\\bruce\\Desktop\\project\\real_estate_info\\'
df_a = pd.read_csv(path+'a_lvr_land_a.csv').drop([0])
df_b = pd.read_csv(path+'b_lvr_land_a.csv').drop([0])
df_e = pd.read_csv(path+'e_lvr_land_a.csv').drop([0])
df_f = pd.read_csv(path+'f_lvr_land_a.csv').drop([0])
df_h = pd.read_csv(path+'h_lvr_land_a.csv').drop([0])


In [75]:
total = [df_a, df_b, df_e, df_f, df_h]
df_all = pd.concat(total).reset_index(drop=True)

**filter_a**

In [76]:
filter_a_1 = (df_all['主要用途']=='住家用')
tmp = df_all.loc[filter_a_1]

In [77]:
tmp = tmp.loc[tmp['建物型態'].str.contains('住宅大樓', na=False)]

因為總樓層數是以中文數字表示，因次必須將其轉換為阿拉伯數字，並去除'層'這個無用字眼

In [78]:
def convert_floor(chinese):
    numbers = {'一':1, '二':2, '三':3, '四':4, '五':5, '六':6, '七':7, '八':8, '九':9}
    units = {'十':10}
    number, pureNumber = 0, True
    for i in range(len(chinese)):
        if chinese[i] in units:
            pureNumber = False
            break
        if chinese[i] in numbers:
            number = number * 10 + numbers[chinese[i]]
    if pureNumber:
        return number
        
    number = 0
    for i in range(len(chinese)):
        base = 10
        if chinese[i] in numbers and len(chinese)==3:
            number = numbers[chinese[i]]*base + numbers[chinese[i+2]]
            break
        elif chinese[i] == '十':
            number = base + numbers[chinese[i+1]]
            break
        elif chinese[i+1] == '十' and len(chinese)==2:
            number = numbers[chinese[i]]*base
            break
    return number

In [79]:
tmp['總樓層數'] = tmp['總樓層數'].str.replace('層','')
tmp['總樓層數'] = tmp['總樓層數'].apply(convert_floor)

In [80]:
filter_a = tmp.loc[tmp['總樓層數']>=13]

In [89]:
filter_a.reset_index(drop=True)
filter_a.to_csv('filter_a.csv')

**filter_b**

In [82]:
total_count = len(df_all)
print(total_count)

8864


In [83]:
total_spot_count = 0
for i in range(len(df_all)):
    num = int(df_all['交易筆棟數'].loc[i][-1])
    total_spot_count += num
print(total_spot_count)

4616


In [84]:
average_price = df_all['總價元'].astype(int).mean()
print(average_price)


15184413.837770758


In [85]:
num = 0
for i in range(len(df_all)):
    if df_all['交易筆棟數'].loc[i][-1] != '0':
        #print(df_all['車位總價元'].loc[i])
        num += int(df_all['車位總價元'].loc[i])
average_spot_price = num/total_spot_count
print(average_spot_price)

1031953.4228769498


In [86]:
d = {'總件數':[total_count],'總車位數':[total_spot_count],\
    '平均總價元':[average_price], '平均車位總價元':[average_spot_price]}
filter_b = pd.DataFrame(data=d,index=['答案'])

In [90]:
filter_b.to_csv('filter_b.csv')